In [ ]:
!pip install --upgrade google-cloud-bigquery
from google.colab import auth
auth.authenticate_user()
from google.cloud import bigquery

In [ ]:
# Set project ID
project_id = "rock-finder-project"
client = bigquery.Client(project=project_id)

In [ ]:
query = """
SELECT * FROM `rock-finder-project.routes.df_routes_gold_multipitches`
"""
df = client.query(query).to_dataframe()

In [ ]:
query = """
SELECT * FROM `rock-finder-project.routes.clusters`
"""
clusters = client.query(query).to_dataframe()

In [ ]:
#Pandas
import pandas as pd

In [ ]:
#Pandas
import pandas as pd

In [ ]:
df.columns

In [ ]:
df_merged = pd.merge(df,clusters, on = "key")

In [ ]:
df_merged.columns

In [ ]:
df_merged[["location_split","key","cluster"]]

In [ ]:
from collections import Counter

# Function to count words in each cluster
def count_words_per_cluster(group):
    # Flatten all the lists of strings into one big list
    all_words = sum(group["location_split"], [])
    return Counter(all_words)

# Group by cluster and apply the word counter
word_counts = df_merged.groupby("cluster").apply(count_words_per_cluster)

In [ ]:
word_counts_df = word_counts.reset_index(name="word_counts")

In [ ]:
df_final = df_merged.merge(word_counts_df, on="cluster", how="left")

In [ ]:
# Get the top words with the counter 
def get_top_words(counter):
    if not isinstance(counter, dict) or not counter:
        return []
    max_count = max(counter.values())
    return [word for word, count in counter.items() if count == max_count]

df_final["top_words"] = df_final["word_counts"].apply(get_top_words)
df_final["top_word_count"] = df_final["word_counts"].apply(lambda x: max(x.values()) if x else None)

In [ ]:
df_final.columns

In [ ]:
df_crag = df_final[['top_words',"cluster",'top_word_count',"key"]]

In [ ]:
#Get only the first item from the top words count (incase there where miltiple words with the same value)
df_crag["crag_name"] = df_crag["top_words"].apply(lambda x: x[0])

In [ ]:
#Remove any non letter character values 
import re
import unicodedata
import pandas as pd

def clean_crag_name(name):
    if pd.isnull(name):
        return ''
    # Normalize to NFKD (decompose accents etc.)
    normalized = unicodedata.normalize('NFKD', name)
    # Remove non-letter characters
    return re.sub(r'[^A-Za-z]', '', normalized)

df_crag["crag_name_clean"] = df_crag["crag_name"].apply(clean_crag_name)

In [ ]:
# define the final crag table 
df_crag_final = df_crag[["key","cluster","crag_name_clean"]]
df_crag_final

In [ ]:
#Prepare to upload new dataframe to Big Query
project_id = "rock-finder-project"
dataset_id = "rock-finder-project.routes"
table_id = "crag_name"
full_table_id = f"{dataset_id}.{table_id}"

In [ ]:
#Upload new Dataframe to Big query 
from pandas_gbq import to_gbq
to_gbq(df_crag_final, destination_table=full_table_id, project_id=project_id, if_exists="replace")